In [1]:
# 导入包
import random
import math
import time
import numpy as np
import pandas as pd
import warnings
import os
import sys
import operator
warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True)

In [2]:
def read_data(path):
    data = []
    with open(path,"r") as f:
        for line in f.readlines():
            user,movie,rating,_ = line.split("::")
            data.append([user, movie])
        return data

In [3]:
def split_data(data, M=5, k=1):
    train = []
    test = []
    random.seed(42)
    for line in data:
        if random.randint(0, M) == k:
            test.append(line)
        else:
            train.append(line)
    return train, test

In [4]:
# convert data to dict format
def transform_data(data):   
    data_dict = {}
    for user, movie in data:
        if user not in data_dict:
            data_dict[user] = set()
        data_dict[user].add(movie)
        
    data_dict = {user : list(data_dict[user]) for user in data_dict}   
    return data_dict        

In [5]:
def preprocess_data(path):
    raw_data = read_data(filePath)
    train_set, test_set = split_data(raw_data)
    train = transform_data(train_set)
    test = transform_data(test_set)
    return train, test

In [6]:
def GetRecommendation(result, user):
    rank = result[user]
    return rank
    
def Recall(train, test, result):
    hit = 0
    all = 0
    for user in train.keys():
        tu = test.get(user)
        if tu is None:
            continue
        rank = GetRecommendation(result, user)
        for item in rank:
            if item in tu:
                hit += 1
        all += len(tu)
    return hit / (all * 1.0)
    
def Precision(train, test, result):
    hit = 0
    all = 0
    for user in train.keys():
        tu = test.get(user)
        if tu is None:
            continue    
        rank = GetRecommendation(result, user)
        for item in rank:
            if item in tu:
                hit += 1
        all += len(rank)
    return hit / (all * 1.0)
    
def Coverage(train, test, result):
    recommend_items = set()
    all_items = set()
    for user in train.keys():
        for item in train[user]:
            all_items.add(item)
        rank = GetRecommendation(result, user)
        for item in rank:
            recommend_items.add(item)
    return len(recommend_items) / (len(all_items) * 1.0)
    
def Popularity(train, test, result):
    item_popularity = dict()
    for user, items in train.items():
        for item in items:
            if item not in item_popularity:
                item_popularity[item] = 0
            item_popularity[item] += 1

    ret = 0
    n = 0
    for user in train.keys():
        rank = GetRecommendation(result, user)
        for item in rank:
            ret += math.log(1 + item_popularity[item])
            n += 1
        ret /= n * 1.0
    return ret

In [7]:
def rec_summary(result, numFold=5):
    precision =0
    recall = 0
    coverage = 0
    popularity =0

    for i in range(0, numFold):
        precision += Precision(train,test, result)
        recall += Recall(train,test,result)
        coverage += Coverage(train, test, result)
        popularity += Popularity(train, test, result)

    precision /= numFold
    recall /= numFold
    coverage /= numFold
    popularity /= numFold

    print('precision = %f' %precision)
    print('recall = %f' %recall)
    print('coverage = %f' %coverage)
    print('popularity = %f' %popularity)

## 2.4.1 基于用户的协同过滤算法

In [8]:
# 余弦相似度, 时间复杂度是O(|U|*|U|)
def UserSimilarityV1(train):
    W = dict()
    for u in train.keys():
        for v in train.keys():
            if u == v:
                continue
            W[u][v] = len(train[u] & train[v])
            W[u][v] /= math.sqrt(len(train[u]) * len(train[v]) * 1.0)
    return W            

In [9]:
# UserCF
def UserSimilarityV2(train):
    # build inverse table for item_users
    item_users = dict()
    for u, items in train.items():
        for i in items:
            if i not in item_users:
                item_users[i] = set()
            item_users[i].add(u)
    
    #calculate co-rated items between users
    C = dict()
    N = dict()
    for i, users in item_users.items():
        for u in users:
            N.setdefault(u,0)
            C.setdefault(u,{})
            N[u] += 1
            for v in users:
                if u == v:
                    continue
                C[u].setdefault(v,0)
                C[u][v] += 1
                
    #calculate finial similarity matrix W
    W = C.copy()
    for u, related_users in C.items():
        for v, cuv in related_users.items():
            W[u][v] = cuv / math.sqrt(N[u] * N[v])
    return W

In [10]:
# Random Recommendation
def RandomRec(train, N):
    all_items = set()
    result = dict()
    for item_list in train.values():
        for item in item_list:        
            all_items.add(item)
     
    for user in train.keys():    
        user_items = train[user]
        rec_items = [item for item in all_items if item not in user_items]
        result[user] = rec_items[:N]
    return result

In [11]:
def MostPopularRec(train, N):
    items = dict()
    result = dict()
       
    for item_list in train.values():
        for item in item_list:        
            if item not in items:
                items[item] = 0
            items[item] += 1            
            
    for user in train.keys():
        user_items = train[user]
        item_list = []
        rec_items = {item: items[item] for item in items if item not in user_items}
        rec_items_list = list(sorted(rec_items.items(), key=operator.itemgetter(1), reverse=True))[:N]
        for tuple in rec_items_list:
            item_list.append(tuple[0])
        result[user] = item_list
    
    return result

In [12]:
def UserCFRec(train, N, K):
    W = UserSimilarityV2(train)
    
    rank = dict()
    result = dict()
    
    for user in train.keys():  
        interacted_items = train[user]
        item_list = []
        for v, wuv in sorted(W[user].items(), key=operator.itemgetter(1), reverse=True)[:K]:
            for v_item in train[v]:                
                if v_item in interacted_items:
                    continue
                rank.setdefault(v_item,0)
                rank[v_item] += wuv
            
        rec_items = list(sorted(rank.items(), key=operator.itemgetter(1), reverse=True)[:N])
        for tuple in rec_items:
            item_list.append(tuple[0])
        result[user] = item_list
    return result

In [20]:
# User-IIF
def UserSimilarityV3(train):
    # build inverse table for item_users
    item_users = dict()
    for u, items in train.items():
        for i in items:
            if i not in item_users:
                item_users[i] = set()
            item_users[i].add(u)
    
    #calculate co-rated items between users
    C = dict()
    N = dict()
    for i, users in item_users.items():
        for u in users:
            N.setdefault(u,0)
            C.setdefault(u,{})
            N[u] += 1
            for v in users:
                if u == v:
                    continue
                C[u].setdefault(v,0)
                C[u][v] += 1 / math.log(1 + len(users))
                
    #calculate finial similarity matrix W
    W = C.copy()
    for u, related_users in C.items():
        for v, cuv in related_users.items():
            W[u][v] = cuv / math.sqrt(N[u] * N[v])
    return W

In [23]:
def UserIIFRec(train, N, K):
    W = UserSimilarityV3(train)
    
    rank = dict()
    result = dict()
    
    for user in train.keys():
        interacted_items = train[user]
        item_list = []
        
        for v, wuv in sorted(W[user].items(), key = operator.itemgetter(1), reverse=True)[:K]:
            for v_item in train[v]:                 
                if v_item in interacted_items:
                    continue
                rank.setdefault(v_item,0)
                rank[v_item] += wuv 
            
        rec_items = list(sorted(rank.items(), key=operator.itemgetter(1), reverse=True)[:N])
        for tuple in rec_items:
            item_list.append(tuple[0])
        result[user] = item_list
    return result

### Expirement for User CF

In [15]:
filePath = "./dataset/ratings.dat"
train, test = preprocess_data(filePath)

In [16]:
random_rec_result = RandomRec(train, 10)

In [17]:
rec_summary(random_rec_result)

precision = 0.005655
recall = 0.002054
coverage = 0.004885
popularity = 0.000555


In [18]:
most_popular_result = MostPopularRec(train, 10)

In [19]:
rec_summary(most_popular_result)

precision = 0.158491
recall = 0.057556
coverage = 0.019810
popularity = 0.001272


In [16]:
usercf_result = UserCFRec(train, 10, 10)

In [18]:
rec_summary(usercf_result)

precision = 0.071542
recall = 0.025981
coverage = 0.016011
popularity = 0.001268


In [24]:
useriif_rec_result = UserIIFRec(train, 10, 10)

In [27]:
rec_summary(useriif_rec_result)

precision = 0.069303
recall = 0.025168
coverage = 0.016554
popularity = 0.001257
